# 1. Instalando o `sits`

## 1.1. Instruções para instalar o `sits` no Kaggle

In [ ]:
library(devtools)

In [ ]:
# Instala sits direto do GitHub
install_github("rolfsimoes/sits@kaggle", dependencies = FALSE)

In [ ]:
# Carrega o sits em memória
library(sits)

Se tudo der certo você deverá uma mensagem como esta: 
```
Loaded sits v0.9.6.
        See ?sits for help, citation("sits") for use in publication.
        See demo(package = "sits") for examples.
```

In [ ]:
# Instala o pacote WTSS
install_github("e-sensing/wtss", dependencies = FALSE)

In [ ]:
# Carrega o WTSS
library(wtss)

Se tudo der certo, você deverá ver uma mensagem como esta: 
```
Loaded wtss v2.2.0.
        See ?wtss for help, citation("wtss") for use in publication.
        See demo(package = "wtss") for examples.
```

## 1.2. Instructions to install in  a local machine

If you want to install sits on your personal machine, you must ensure that all system dependencies are installed. The `sits` package relies on `sf` and `rgdal`, which in turn, require the installation of *GDAL* and *PROJ* libraries.

We advise users to try to install these packages first before looking for the instructions to install them. The commands to install are:
```
install.packages("sf")
install.packages("rgdal")
```

After install `sf` and `rgdal` packages, the user can use the following script to install other dependencies:

```
# Install devtools, rmarkdown, knitr, testthat and Rcpp 
# if not already installed.
install.packages(c("devtools", "rmarkdown", "Rcpp", 
                   "knitr", "testthat"))

# Install the Suggested packages that are used by sits
install.packages(c("DBI","dendextend", "dtwclust",
                   "dtwSat", "e1071", "flexclust",
                   "imager", "imputeTS", "kohonen", 
                   "lwgeom", "MASS", "mgcv", "nnet", 
                   "proto", "proxy", "ptw", "ranger", 
                   "RCurl", "RSQLite", "signal", 
                   "xgboost", "zoo"))

# Install the Keras package from the RStudio repository
devtools::install_github("rstudio/reticulate")
devtools::install_github("rstudio/keras")

# Build the keras environment
library(keras)
keras::install_keras()

# Install WTSS
devtools::install_github("e-sensing/wtss")
library(wtss)

# Install sits
devtools::install_github("e-sensing/sits")
library(sits)

```

After these steps, your machine should have the latest version of `sits` working.
 

# 2. Acesso a Séries Temporais com o sits

## 2.1. Acesso a provedores de séries temporais

In [ ]:
# Cria uma referencia ao servico WTSS do INPE
wtss <- sits_cube(type = "WTSS",
                  URL = "http://www.esensing.dpi.inpe.br/wtss",
                  name = "MOD13Q1")

# Obtem uma serie temporal nas coordenadas indicadas
ts1 <- sits_get_data(cube = wtss,
                     longitude = -55.57320,
                     latitude = -11.50566)

In [ ]:
print(ts1)


## 2.2. Formato de arquivo CSV para acesso em lote a séries temporais

In [ ]:
cat(readLines("../input/sits-dataset/examples.csv"), fill = TRUE)

In [ ]:
ts2 <- sits_get_data(cube = wtss,
                     file = "../input/sits-dataset/examples.csv")

In [ ]:
print(ts2)

## 2.3. Salvando e lendo tabelas de amostras

In [ ]:
# Salva a tabela no arquivo './tabela.rds'
saveRDS(ts2, "tabela.rds")

# Lê o arquivo e armazena na variável `ts3`
ts3 <- readRDS("tabela.rds")

In [ ]:
print(ts3)

In [ ]:
# Lê o arquivo de amostras e armazena na variável `samples`
samples <- readRDS("../input/sits-dataset/samples.rds")

In [ ]:
print(samples)

## 2.4. Obtendo informações de uma tabela de amostras

In [ ]:
# Quais bandas estão presentes?
sits_bands(samples)

In [ ]:
# Quantas amostras existem em cada classe?
sits_labels(samples)

## 2.5. Filtrando uma tabela de amostras

In [ ]:
# carrega a biblioteca `dplyr`
library(dplyr)

In [ ]:
# Filtra as amostras e mostra as linhas da tabela 
print(filter(samples, label == "Forest"))

In [ ]:
# Copia a tabela apenas com a banda 'ndvi'
samples_ndvi <- sits_select(samples, bands = "ndvi")

In [ ]:
# Mostra as bandas presentes na tabela de amostras
sits_bands(samples_ndvi)

In [ ]:
# Configura tamanho de saída dos gráficos
options(repr.plot.width = 10, repr.plot.height = 3)

In [ ]:
# Gera gráfico da série temporal `ts1`
plot(ts1)

In [ ]:
# Gera gráfico da primeira série temporal em `samples`
plot(head(samples, 1))

In [ ]:
# Gera os gráficos das séries temporais de NDVI das classes separadamente
plot(samples_ndvi)

# 3. Classificação de Séries Temporais

## 3.1. Exemplo Random Forests

In [ ]:
# Treina um modelo RF com 200 árvores usando as bandas
# 'ndvi', 'evi', 'nir' e 'mir'
rf1 <- sits_train(samples,
                  ml_method = sits_rfor(num_trees = 200))

# Treina um modelo RF com 200 árvores usando a banda 'ndvi'
rf2 <- sits_train(samples_ndvi,
                  ml_method = sits_rfor(num_trees = 200))

In [ ]:
# rf1: modelo RF bandas 'ndvi', 'evi', 'nir' e 'mir'
ts_4bands <- sits_select(ts1, 
                         bands = c('ndvi', 'evi', 'nir', 'mir'))

# Classifica série temporal
rf_class1 <- sits_classify(ts_4bands, ml_model = rf1)

In [ ]:
# rf2: modelo RF banda 'ndvi'
ts_ndvi <- sits_select(ts1, bands = c('ndvi'))

# Classifica série temporal
rf_class2 <- sits_classify(ts_ndvi, ml_model = rf2)

In [ ]:
plot(rf_class1)

In [ ]:
plot(rf_class2)

## 3.2. Exemplo Deep Learning MLP

In [ ]:
# Treina um modelo MLP com 5 camadas escondidas, 512 unidades,
#  função de ativação 'elu', bandas 'ndvi'
dl1 <- sits_train(samples_ndvi,
                  ml_method = sits_deeplearning(epochs = 200))

In [ ]:
# dl1: modelo MLP bandas 'ndvi'
# Classifica série temporal
dl_class1 <- sits_classify(ts_ndvi, ml_model = dl1)

In [ ]:
plot(dl_class1)

## 3.3 Consultando a Documentação

In [ ]:
?sits_svm

In [ ]:
# Exemplo extraído da documentação da função `sits_svm()`:


# Retrieve the set of samples for  Mato Grosso  (provided by EMBRAPA)
samples_2bands <- sits_select(samples_mt_6bands, bands = c("ndvi", "evi"))

# Build a machine learning model based on deep learning
ml_model <- sits_train (samples_2bands, sits_svm())

# get a point and classify the point with the ml_model
point.tb <- sits_select(point_mt_6bands, bands = c("ndvi", "evi"))
class.tb <- sits_classify(point.tb, ml_model)
plot(class.tb, bands = c("ndvi", "evi"))


# 4. Avaliação da qualidade das amostras usando SOM

# 5. Classificação de *bricks* e geração de mapas

In [ ]:
# Carrega timeline do arquivo
timeline <- readLines("../input/sits-dataset/timeline.txt")

timeline

In [ ]:
files <- c("../input/sits-dataset/NDVI.tif", 
           "../input/sits-dataset/EVI.tif")
files

In [ ]:
sinop <- sits_cube(type = "BRICK",
                   name = "sinop-mt", 
                   satellite = "TERRA",
                   sensor = "MODIS",
                   timeline = timeline,
                   bands = c("ndvi", "evi"), 
                   files = files)

In [ ]:
# Cria um novo conjunto de amostras apenas com as bandas 'ndvi' e 'evi' 
samples_2bands <- sits_select(samples, bands = sits_bands(sinop))

# Treina um modelo RF com 200 árvores
rf3 <- sits_train(samples_2bands, ml_method = sits_rfor(num_trees = 200))

In [ ]:
# Classifica o cubo `sinop`
probs <- sits_classify(sinop, ml_model = rf3)

In [ ]:
# Configura tamanho de saída dos gráficos
options(repr.plot.width = 18, repr.plot.height = 9)

# Mostra mapas de probabilidades
plot(probs)

In [ ]:
# Gera mapa de uso e cobertura da terra
maps <- sits_label_classification(probs)

In [ ]:
# Configura tamanho de saída dos gráficos
options(repr.plot.width = 12, repr.plot.height = 10)

# Mostra mapa de uso e cobertura da terra
plot(maps)

# 6. Estimação de acurácia 

In [ ]:
# Realiza uma avaliação de acurácia de um modelo Random Forest 
# usando parâmetros default. k=5
prediction.mx <- sits_kfold_validate(samples_2bands, 
                                     folds = 5, 
                                     ml_method = sits_rfor(num_trees = 200))

# Mostra a matriz de confusão e métricas de acurácia 
sits_conf_matrix(prediction.mx)